In [ ]:
import concurrent.futures
from functools import partial
import json
import os
import pandas as pd
import pyperclip
import numpy as np
import random
import re
import requests
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
from user_agent import generate_user_agent, generate_navigator

In [ ]:
# Настраиваем подмену user agent, выключаем загрузку картинок
ua = generate_user_agent(os=('mac', 'linux'))
opts = Options()
opts.add_argument("user-agent={}".format(ua))

chrome_prefs = {}
opts.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

In [ ]:
# Запасные сайты с proxy:
# https://openproxy.space/list
# https://www.sslproxies.org
# http://www.freeproxylists.net/ru/
# https://www.proxynova.com/proxy-server-list

# Каждая функция собирает proxy с соответствующего сайта и возвращает лист

def get_proxy_spysone():
    try:
        url = 'http://spys.one/en/'
        driver = webdriver.Chrome(options=opts)
        driver.get(url)
        tags = driver.find_elements_by_class_name('spy14')
        proxies = []
        for tag in tags:
            if re.search('\d+\.\d+\.\d+\.', tag.text):
                proxies.append(tag.text)
        driver.close()
        driver.quit()
        return proxies
    except:
        return []

# похоже этот сайт умер.....
def get_proxies_freeproxycz():
    try:
        url = 'http://free-proxy.cz/en/proxylist/main/1'
        driver = webdriver.Chrome(options=opts)
        proxies = []
        for i in range(1, 6):
            driver.get(url[:-1] + str(i))
            td_tags = driver.find_elements_by_tag_name('td')
            for td_tag in td_tags:
                if re.search('\d+\.\d+\.\d+\.', td_tag.text):
                    proxies.append(td_tag.text)
        driver.quit()
        return proxies
    except:
        return []


def get_proxies_proxyscrapecom():
    try:
        url = 'https://proxyscrape.com/free-proxy-list'
        driver = webdriver.Chrome(options=opts)
        driver.get(url)
        time.sleep(10)
        td_tags = driver.find_element_by_id('proxytable').find_elements_by_tag_name('td')
        proxies = []
        for td_tag in td_tags:
            if re.search('\d+\.\d+\.\d+\.', td_tag.text):
                    proxies.append(td_tag.text)
        driver.close()
        return proxies
    except:
        return []

def get_proxies_proxylistdownload():
    try:
        url = 'https://www.proxy-list.download/HTTP'
        driver = webdriver.Chrome(options=opts)
        driver.get(url)
        time.sleep(5)
        driver.find_element_by_id('btn3').click()
        alert = driver.switch_to.alert
        alert.accept()
        proxies = pyperclip.paste().split('\n')
        driver.close()
        return proxies
    except:
        return []

def get_proxies_ipadresscom():
    try:
        url = 'https://www.ip-adress.com/proxy-list'
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        td_tags = soup.find('tbody').find_all('td')
        proxies = [td.text for td in td_tags if td.find('a') != None]
        return proxies
    except:
        return []

def get_proxies_hidemyname():
    try:
        url = 'https://hidemy.name/ru/proxy-list/?type=h&start=1#list'
        pages = [1, 64]
        proxy = []
        for p in pages:
            string = proxy_url2
            url = string[:-6] + str(p) + string[-5:]
            driver = webdriver.Chrome(options=opts)
            driver.get(url)
            time.sleep(10)
            for tr in driver.find_elements_by_tag_name('tr')[1:]:
                proxy = tr.find_elements_by_tag_name('td')[0].text + ':' + \
                tr.find_elements_by_tag_name('td')[1].text
                proxies.append(proxy)
            driver.close()
        return proxies
    except:
        return []

# подаем функции выше на вход - на выходе объединенный лист "сырых" proxy 
def create_raw_proxylist(*args):
    raw_proxy_list = []
    for func in args:
        raw_proxy_list += func()
    return raw_proxy_list

# бьем лист "сырых" proxy на куски (нужно для multi-threading)
def make_proxylist_chunks(raw_proxy_list, chunk_size):
    list_len = len(raw_proxy_list)
    chunks = []
    for i in range(0, list_len, chunk_size):
        chunk = raw_proxy_list[i:i+chunk_size]
        chunks.append(chunk)
    return chunks

# фильтруем proxy из "куска сырого" proxy-листа на работоспособность - ходим на mail.ru (любой сайт можно впилить) 
# и добавлем в чистый список, что ошибок не будет. Возвращаем чистый список
def filter_raw_proxylist(chunk):
    check_url = 'https://auto.ru'
    filtered_proxies_chunk = []
    for proxy in chunk:
        try:
            r = requests.get(check_url, 
                            headers = {'User-Agent':generate_user_agent(os=('mac', 'linux')),
                                      'Referer': 'https://mail.ru/'}, 
                            proxies = {'http': 'http://'+ proxy, 'https':'http://' + proxy},
                            timeout=12)
            r.encoding = r.apparent_encoding
            soup = BeautifulSoup(r.text)
# если ходить на auto.ru - нужно проверять на предмет выдачи страницы про перс данные
#             try:
#                 if 'обработка моих данных' in soup.find('div', class_='text').text:
#                     pass    
#             except:
            filtered_proxies_chunk.append(proxy)
        except requests.exceptions.RequestException as e:
            pass
    return filtered_proxies_chunk

# функция для выбора рандомного proxy из листа/куска
def get_random_proxy(proxies):
    return random.choice(proxies)

In [ ]:
# получаем html
def get_soup(url, proxies):
    proxy = get_random_proxy(proxies)
    r = requests.get(url, 
                     headers = {'User-Agent':generate_user_agent(os=('mac', 'linux'))}, 
                     proxies = {'http': 'http://'+proxy, 
                                'https':'http://'+proxy},
                     timeout=12)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, "html.parser")
    return soup

# получаем html на proxy из листа. Если ошибка - удаляем proxy из листа и рекурсивно повторяем, пока лист
# proxy не опустошится. Вообще изначально на этом функция выдавала None - но пока поставил рискованное решение - 
# попробовать в случае опустошения proxy листа зайти без proxy. На 100 000 url сработало, но это, наверно, плохой
# подход
def get_html(url, proxies):
    try:
        html = get_soup(url, proxies)
        return html
    except:
        proxy = get_random_proxy(proxies)
        proxies.remove(proxy)
#         print(f'Удален proxy {proxy}, осталось {len(proxies)}')
        if proxies == []:
            print('Все proxy битые')
            r = requests.get(url)
            return BeautifulSoup(r.text, 'html.parser')
        html = get_html(url, proxies)
        return html

# делаем куски из листа ссылок на объявления о продаже
def make_links_chunks(links_list, chunk_size):
    list_len = len(links_list)
    chunks = []
    for i in range(0, list_len, chunk_size):
        chunk = links_list[i:i+chunk_size]
        chunks.append(chunk)
    return chunks

In [ ]:
# собираем данные с блока "card_info" слева от фотографий
def get_main_info(info_dict, offer_soup):
    card_info = offer_soup.find('ul', class_='CardInfo')
    card_info_classes = [
    "CardInfo__row_year",
    "CardInfo__row_kmAge",
    "CardInfo__row_bodytype",
    "CardInfo__row_color",
    "CardInfo__row_engine",
    "CardInfo__row_transportTax",
    "CardInfo__row_transmission",
    "CardInfo__row_drive",
    "CardInfo__row_wheel",
    "CardInfo__row_state",
    "CardInfo__row_ownersCount",
    "CardInfo__row_pts",
    "CardInfo__row_owningTime",
    "CardInfo__row_customs",
    "CardInfo__row_vin",
    "CardInfo__row_licensePlate",
    ]
    
    # цикл сбора 
    for class_ in card_info_classes:
        param_name = class_.split('_')[-1]
        try:
            li_tag = card_info.find('li', class_=class_)
            if li_tag == None:
                info_dict.update({param_name: []})
                info_dict[param_name].append(None)
                continue
            param_value = li_tag.find_all('span')[-1].text
            if param_name not in info_dict.keys():
                info_dict.update({param_name: []})
                info_dict[param_name].append(param_value)
        except:
            info_dict[param_name].append(None)

# рудимент прошлых попыток - оставил для одного сборщика. Идея - везде в функции пихать try/except неправильно,
# поэтому сделал обертку. Но есть функция, ниже (get_reviews_features), которая в эту обертку не залезает - но
# ошибку нужно обходить. Поэтому оставил, но вообще эту функцию при оптимизации лучше убрать.
def get_info_safely(func):
    def func_wrapper(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
        except Exception as err:
            print(func.__name__)
            print(err)
            return None
        return result
    return func_wrapper

# берем название модели
def get_model_name(info_dict, offer_soup):
    try:
        price = offer_soup.find('div', class_="CardSidebarActions__title").text
    except:
        price = None
    key='model_name'
    if key not in info_dict.keys():
        info_dict.update({key: []})   
    info_dict[key].append(price)
    
# забираем цену
def get_price_value(info_dict, offer_soup):
    try:
        price = offer_soup.find('span', class_='OfferPriceCaption__price').text
    except:
        price = None
    key='price'
    if key not in info_dict.keys():
        info_dict.update({key: []})   
    info_dict[key].append(price)

# забираем изменение цены (далее не используется, так как очень у малого кол-ва объявлений это поле непустое)
def get_price_change(info_dict, offer_soup):
    try:
        price_change = offer_soup.find('div', class_='OfferPriceDiff__diff').text
    except:
        price_change = None
    key  ='price_change'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(price_change)

# берем дату публикации
def get_publication_date(info_dict, offer_soup):
    try:
        div_tags = offer_soup.find_all('div')
        publication_date = [div for div in div_tags if div.has_attr('title')==True][0].text
    except:
        publication_date = None
    key = 'publication_date'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(publication_date)

# берем количество просмотров (далее не используется, так как очень у малого кол-ва объявлений это поле непустое)
def get_offer_views(info_dict, offer_soup):
    try:
        div_tags = offer_soup.find_all('div')
        views = [div for div in div_tags if div.has_attr('title')==True][1].text
    except:
        views = None
    key = 'views'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(views)

# берем статус продавца как официального дилера  (далее не используется, 
# так как очень у малого кол-ва объявлений это поле непустое)
def get_offdealer_info(info_dict, offer_soup):
    try:
        offdealer_info = offer_soup.find('div', class_='CardDealerInfo__salonVerifiedLabel').text
    except:
        offdealer_info = ''
    key = 'official_dealer'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(offdealer_info)

# берем рейтинги (далее не используется, 
# так как очень у малого кол-ва объявлений это поле непустое)
def get_reviews_rating(info_dict, offer_soup):
    regex = re.compile(r'"avg_rating":\d+.\d+,"counter":\d+')
    try:
        match = regex.search(str(offer_soup))[0]
        reviews_rating = match.split(',')[0].split(':')[1]
    except:
        reviews_rating = 0
    key = 'reviews_avg_rating'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(reviews_rating)

# берем количество обзоров (далее не используется, 
# так как очень у малого кол-ва объявлений это поле непустое)
def get_reviews_count(info_dict, offer_soup):
    regex = re.compile(r'"avg_rating":\d+.\d+,"counter":\d+')
    try:
        match = regex.search(str(offer_soup))[0] 
        reviews_count = match.split(',')[1].split(':')[1]
    except:
        reviews_count = 0
    key = 'reviews_count'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(reviews_count)

# берем оценки параметров модели (далее не используется, 
# так как очень у малого кол-ва объявлений это поле непустое)
@get_info_safely    
def get_reviews_features(info_dict, offer_soup):
    regex = re.compile(r'"reviewsFeatures":{.*{.*{.*\[{.*}]}}}')
    string = '{' + regex.search(str(offer_soup))[0] + '}'
    reviews_dict = json.loads(string, encoding='utf-8')
    keys = reviews_dict['reviewsFeatures']['data']['features'].keys()
    for key in keys:
        temp_dict = reviews_dict['reviewsFeatures']['data']['features'][key]
        for item in temp_dict:
            name_pos = item['type'] + '_pos'
            name_neg = item['type'] + '_neg'
            name_ratio = item['type'] + '_ratio'
            val_pos = item['plus_count']
            val_neg = item['minus_count']
            val_ratio = item['plus_count']/item['total_count']
            for dict_key in (name_pos, name_neg, name_ratio):
                if dict_key not in info_dict.keys():
                    dict_val_len = len(info_dict['year'])
                    info_dict.update({dict_key: ['']*(info_dict_len-1)})
            info_dict[name_pos].append(val_pos)
            info_dict[name_neg].append(val_neg)
            info_dict[name_ratio].append(val_ratio)

# подсчитываем количество фотографий
def get_photo_count(info_dict, offer_soup):
    try:
        photo_count = len(offer_soup.find('div', class_="ImageGalleryDesktop__thumbs-container").
                          find_all('div'))
    except:
        photo_count = 0
    key = 'photo_count'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(photo_count)

# подсчитываем количество элементов комплектации (далее не используется, 
# так как очень у малого кол-ва объявлений это поле непустое)
@get_info_safely
def get_complectation_count(info_dict, offer_soup):
    div_tags = offer_soup.findAll('div', class_="CardComplectation__group")
    complectation_names = [div.find('span', class_="CardComplectation__itemName").text 
                           for div in div_tags]
    complectation_counts = [int(div.find('span', class_='CardComplectation__itemCount').text)
                           for div in div_tags]
    
    for complectation_name, complectation_count in zip(complectation_names,
                                                        complectation_counts):
        key = 'Комплектация_' + complectation_name
        if key not in info_dict.keys():
            info_dict.update({dict_key: ['']*(info_dict_len-1)})
        info_dict[key].append(complectation_count)

# ищем информацию о том, что машина на гарантии
def get_warranty_info(info_dict, offer_soup):
    try:
        descr = offer_soup.find('div', class_="CardDescription__textInner").text
    except:
        descr = ' '
    regex = re.compile('гаранти[ия]')
    key = 'on_warranty'
    warranty = int((re.search(regex, descr.lower()) != None))
    if key not in info_dict.keys():
        info_dict.update({dict_key: ['']*(info_dict_len-1)})
    info_dict[key].append(warranty)

# ищем информацию о дтп
def get_crashes_info(info_dict, offer_soup):
    try:
        div_tags = offer_soup.find_all('div', class_="CardBenefits__item-title")
        no_crashes = 0
        for div in div_tags:
            if 'ДТП не найдены' in div.text:
                no_crashes = 1
    except:
        no_crashes = 0
    key = 'no_crashes'
    if key not in info_dict.keys():
        info_dict.update({key: []})
    info_dict[key].append(no_crashes)

In [ ]:
# считываем ссылки на объявления, скачанные в отдельные файлы на прошлом этапе в DataFrame
data = pd.DataFrame()
for file in os.listdir('../../../Desktop/Autoru/'):
    if file.endswith('.xlsx'):
        df = pd.read_excel('~/Desktop/Autoru/' + file, headers=None)
        data = pd.concat([data, df])

In [ ]:
# делаем куски по 150 ссылок для мульти-трединга
links_list = list(data[0].values)
links_chunks = make_links_chunks(links_list, chunk_size=150)
print(len(links_chunks))

In [ ]:
# словарь, в который будут падать данные. Структура {'url объявления': {'параметр': значение}}
# сделано так, чтобы мульти-трединг работал по каждой ссылке отдельно - иначе одни и те же
# параметры и значения могут затираться тем или иным тредом
data_dict = {}

In [ ]:
# финальный сборщик данных
def get_info(url, data_dict, proxies):  
    data_dict.update({url:{}})
    info_dict = data_dict[url]
    offer_soup = get_html(url, proxies)
    get_main_info(info_dict, offer_soup)
    get_model_name(info_dict, offer_soup)
    get_price_value(info_dict, offer_soup)
    get_price_change(info_dict, offer_soup)
    get_publication_date(info_dict, offer_soup)
    get_offer_views(info_dict, offer_soup)
    get_photo_count(info_dict, offer_soup)
    get_offdealer_info(info_dict, offer_soup)
#     get_complectation_count(info_dict, offer_soup)
    get_warranty_info(info_dict, offer_soup)
    get_crashes_info(info_dict, offer_soup)
#     get_reviews_rating(info_dict, offer_soup)
#     get_reviews_count(info_dict, offer_soup)
#     get_reviews_features(info_dict, offer_soup)

In [ ]:
# создаем "сырой" proxy-лист (можно убрать этот блок, делалось, чтобы следующую ячейку можно было тестировать,
# не запуская каждый раз сборку чистого proxy-листа - это делается несколько минут)
raw_proxy_list = create_raw_proxylist(get_proxy_spysone, get_proxies_freeproxycz,
                         get_proxies_proxyscrapecom, get_proxies_proxylistdownload,
                         get_proxies_ipadresscom, get_proxies_hidemyname)
# размер куска для мульти-тред проверки
chunk_size = 10
# делаем куски "сырого" листа
chunks = make_proxylist_chunks(raw_proxy_list, chunk_size)

# проверяем все куски в отдельном треде (около 100 тредов одновременно)
proxies = []
for _ in range(4):
    with concurrent.futures.ThreadPoolExecutor() as ex:
        filtered_proxies_chunks = ex.map(filter_raw_proxylist, chunks)
        for filtered_chunk in filtered_proxies_chunks:
            proxies += filtered_chunk
print(len(proxies))

In [ ]:
# основной блок
len_proxies = len(proxies)

# для каждого куска листа ссылок на объявления
for i, chunk in tqdm(enumerate(links_chunks)):
# если что-то сломалось на каком-то шаге - можно пропустить выполненные
#     if i < 14:
#         continue

# если чистых proxy меньше 100 штук в листе (обычно столько хватает на обработку 150 ссылок 150ю тредами)-
# запускам сбор proxy листа
    if len_proxies < 100:
        raw_proxy_list = create_raw_proxylist(get_proxy_spysone, get_proxies_freeproxycz,
                                 get_proxies_proxyscrapecom, get_proxies_proxylistdownload,
                                 get_proxies_ipadresscom, get_proxies_hidemyname)
        chunk_size = 10
        chunks = make_proxylist_chunks(raw_proxy_list, chunk_size)
# нужно 4 запуска, чтобы собрать 100 полу-живых proxy
        for _ in range(4):
            with concurrent.futures.ThreadPoolExecutor() as ex:
                filtered_proxies_chunks = ex.map(filter_raw_proxylist, chunks)
                for filtered_chunk in filtered_proxies_chunks:
                    proxies += filtered_chunk
    print(f'Len proxies = {len_proxies}')

# основной блок сбора информации - берем кусок листа ссылок (chunk), и, пользуюясь чистым proxy-листом
# собираем данные по объявлениям
# этот цикл работает около 50 часов
    with concurrent.futures.ThreadPoolExecutor() as ex:
        ex.map(partial(get_info, data_dict=data_dict, proxies = proxies), chunk)
# смотрим, сколько "живых" proxy осталось
    len_proxies = len(proxies)
# каждые 5*len(chunk) обработанных объявлений (750 в данном случае, изначально было 1000, но не хватало proxy)
# сохраняем info_dict в json на жесткий диск (накопительный файл, последний содержит всю предыдущую информацию)
    if (i+1) % 5 == 0:
        number = str(i+1)
        with open('../../../Desktop/data/data_' + str(i) + '.json', 'w', encoding='mac_cyrillic') as file:
            json.dump(data_dict, file)
    print(f'{i+1} chunk finished')